In [38]:
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [39]:
log_train_df= log_train
train_df = train

In [45]:
log_train.head()

,ID,time,source,event,object,hour,day_of_week,day,time_diff
0,43980,2013-11-11 16:30:11,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,16,0,11,49.0
1,2648,2013-12-27 06:44:08,browser,access,UaqaZdJXEAHfzB9qbZkvAe2jc0xmI3Na,6,4,27,2.0
2,18588,2013-12-07 09:58:44,server,navigate,l0kj1vWrjGxi2YqQWGitPXDEoH5G5RYd,9,5,7,1.0
3,2679,2014-01-05 13:51:34,browser,access,S64EUU2p7wiyAtSxjzfSS6jLaHB3Y9RJ,13,6,5,1.0
4,2528,2013-12-17 12:09:36,server,access,DSPbj5O8mNBS0ccEiBGXnXJvTZstHNED,12,1,17,13.0


In [50]:
predtest=0.51
predtest.round()

AttributeError: 'float' object has no attribute 'round'

In [40]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')

# Fill missing values for IDs that did not have event data


train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [41]:
train_df_enriched.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X134,X135,X136,X137,X138,X139,label,event_count,unique_sources,avg_time_diff
0,7054.0,2.015152,0.0,32.0,NaN,4.0,NaN,NaN,7.0,6.0,...,1.0,1.0,1.0,3.0,2.0,NaN,1,0.0,0.0,0.000000
1,147131.0,1.000000,NaN,2.0,NaN,NaN,0.0,NaN,6.0,5.0,...,3.0,1.0,1.0,2.0,10.0,NaN,1,1.0,1.0,0.000000
2,132068.0,1.000000,NaN,1.0,NaN,2.0,NaN,0.0,9.0,1.0,...,1.0,1.0,2.0,2.0,4.0,NaN,1,0.0,0.0,0.000000
3,85195.0,2.071429,0.0,1.0,NaN,2.0,NaN,NaN,8.0,7.0,...,2.0,2.0,1.0,4.0,2.0,NaN,1,40.0,2.0,1182.717949
4,191948.0,2.000000,NaN,2.0,0.0,2.0,NaN,NaN,14.0,2.0,...,2.0,2.0,1.0,2.0,6.0,0.0,1,0.0,0.0,0.000000


In [42]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import SelectFromModel
from lightgbm import early_stopping

# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'median' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

# Define improved model parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.01,  # Lower learning rate
    'num_leaves': 40,  # Slightly higher num_leaves
    'max_depth': -1,
    'min_child_samples': 30,  # Adjust to control over-fitting
    'max_bin': 255,
    'subsample': 0.8,  # Higher subsample
    'subsample_freq': 1,
    'colsample_bytree': 0.8,  # Higher colsample_bytree
    'min_child_weight': 0.001,
    'subsample_for_bin': 200000,
    'min_split_gain': 0.0,
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 0.1,  # L2 regularization
    'verbose': -1
}

# Cross-validation and model training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
predictions = np.zeros(len(X_resampled))
feature_importance_df = pd.DataFrame()

for fold_, (train_index, test_index) in enumerate(skf.split(X_resampled, y_resampled)):
    print(f"Fold {fold_}")
    X_train, X_valid = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_valid = y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)
    
    # Train with early stopping
    clf = lgb.train(params, dtrain, 1000, valid_sets=[dtrain, dvalid], callbacks=[early_stopping(stopping_rounds=100)])
    
    # Feature importance for later analysis
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X.columns
    fold_importance_df["Importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["Fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions[test_index] = clf.predict(X_valid, num_iteration=clf.best_iteration)

# Optional: Feature Selection based on importance
# This step can be adjusted or iterated upon depending on results
threshold = np.percentile(feature_importance_df["Importance"], 75)  # Example threshold
important_features = feature_importance_df[feature_importance_df["Importance"] > threshold]["Feature"].unique()
X_important_features = X_resampled[important_features]

# You could repeat the model training here with X_important_features to see if performance improves

# Final Model evaluation
accuracy = accuracy_score(y_resampled, predictions.round())
print(f"Accuracy: {accuracy:.2f}")
print(confusion_matrix(y_resampled, predictions.round()))
print(classification_report(y_resampled, predictions.round()))


Fold 0


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


Fold 1
Fold 2
Fold 3
Fold 4
Accuracy: 0.92
[[61155  7086]
 [ 3780 63108]]
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     68241
           1       0.90      0.94      0.92     66888

    accuracy                           0.92    135129
   macro avg       0.92      0.92      0.92    135129
weighted avg       0.92      0.92      0.92    135129



In [43]:
from sklearn.metrics import accuracy_score,f1_score
predicted_labels = np.round(predictions).astype(int)
print('F1_Score :',f1_score(y_resampled,predicted_labels))
#F1_Score : 0.9221600231637873 
#F1_Score : 0.9229871645274212 - 0.1 median
#F1_Score : 0.9242930498300427 - mean lightbgm
#F1_Score : 0.9243412961382085 - median lightbgm
#F1_Score : 0.9246502317282795 - lightbgm median learning rate= 0.01
#F1_Score : 0.9207335755241388 - lightbgm median learning rate= 0.01 dart

F1_Score : 0.9207335755241388


In [44]:
from sklearn.metrics import roc_auc_score

# true_labels = ... # This should be your array of true binary labels (0 or 1)
# predicted_probs = ... # This should be your array of predicted probabilities for the positive class

# Calculate AUC
auc_score = roc_auc_score(y_resampled, predicted_labels)

print(f'AUC Score: {auc_score}')
#AUC Score: 0.9234141033971461 - lightbgm median learning rate= 0.01
#AUC Score: 0.9198248761840062 - lightbgm median learning rate= 0.01 dart

AUC Score: 0.9198248761840062


In [36]:
from joblib import dump

# Assuming gbc is your GradientBoostingClassifier instance
dump(clf, '/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Models/light_gbm_0.01LR.joblib')

['/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Models/light_gbm_0.01LR.joblib']

In [ ]:
#from joblib import load

#loaded_model = load('/content/drive/MyDrive/Hackathon_NEU/gbc_AD.joblib')

In [25]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN
from lightgbm import early_stopping


# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'mean' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

# Define model parameters (customize as needed)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'max_bin': 255,
    'subsample': 0.6,
    'subsample_freq': 0,
    'colsample_bytree': 0.3,
    'min_child_weight': 5,
    'subsample_for_bin': 200000,
    'min_split_gain': 0,
    'reg_alpha': 0,
    'reg_lambda': 0,
    'verbose': 0
}

# Cross-validation and model training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
predictions = np.zeros(len(X_resampled))
feature_importance_df = pd.DataFrame()

for fold_, (train_index, test_index) in enumerate(skf.split(X_resampled, y_resampled)):
    print(f"Fold {fold_}")
    X_train, X_valid = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_valid = y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)
    
    clf = lgb.train(params, dtrain, 10000, valid_sets=[dtrain, dvalid],callbacks=[early_stopping(stopping_rounds=100)])
    
    # Feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X.columns
    fold_importance_df["Importance"] = clf.feature_importance()
    fold_importance_df["Fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions[test_index] = clf.predict(X_valid, num_iteration=clf.best_iteration)

# Model evaluation
accuracy = accuracy_score(y_resampled, predictions.round())
print(f"Accuracy: {accuracy:.2f}")
print(confusion_matrix(y_resampled, predictions.round()))
print(classification_report(y_resampled, predictions.round()))


Fold 0
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[220]	training's binary_logloss: 0.167496	valid_1's binary_logloss: 0.20321
Fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[203]	training's binary_logloss: 0.170887	valid_1's binary_logloss: 0.205039
Fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[239]	training's binary_logloss: 0.165746	valid_1's binary_logloss: 0.202623
Fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[205]	training's binary_logloss: 0.17047	valid_1's binary_logloss: 0.20303
Fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[233]	training's binary_logloss: 0.165603	valid_1's binary_logloss: 0.203537
Accuracy: 0.92
[[60787  7454]
 [ 3004 63884]]
              precision    recall  f1-score   suppor

In [26]:
from sklearn.metrics import accuracy_score,f1_score
predicted_labels = np.round(predictions)
print('F1_Score :',f1_score(y_resampled,predicted_labels))
#F1_Score : 0.9221600231637873 
#F1_Score : 0.9229871645274212 - 0.1 median
#F1_Score : 0.9242930498300427 - mean lightbgm
#F1_Score : 0.9243412961382085 - median lightbgm

F1_Score : 0.9243412961382085


In [18]:
log_test_df=log_test
test_df = test

In [20]:
log_test_df['time'] = pd.to_datetime(log_test_df['time'])

# Extract features
log_test_df['hour'] = log_test_df['time'].dt.hour
log_test_df['day_of_week'] = log_test_df['time'].dt.dayofweek
log_test_df['day'] = log_test_df['time'].dt.day

# Count of events per ID
event_count = log_test_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_test_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
test_df_enriched = test_df.merge(event_count, on='ID', how='left')
test_df_enriched = test_df_enriched.merge(unique_sources, on='ID', how='left')



# Time difference features (e.g., average time between events for each ID)
log_test_df['time_diff'] = log_test_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_test_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')



test_df_enriched = test_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
test_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, 'avg_time_diff':0}, inplace=True)

In [21]:
test_df_enriched.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X133,X134,X135,X136,X137,X138,X139,event_count,unique_sources,avg_time_diff
0,17547.0,1.272727,0.0,1.0,0.0,3.0,0.0,0.0,7.0,26.0,...,0.0,2.0,6.0,2.0,4.0,3.0,NaN,2.0,2.0,124.00
1,140449.0,1.000000,0.0,1.0,NaN,1.0,NaN,0.0,8.0,4.0,...,0.0,3.0,2.0,2.0,2.0,5.0,NaN,4.0,1.0,37.00
2,182658.0,2.000000,NaN,2.0,NaN,2.0,NaN,0.0,64.0,2.0,...,NaN,2.0,4.0,1.0,1.0,7.0,0.0,2.0,2.0,155.00
3,149652.0,1.000000,0.0,1.0,NaN,1.0,0.0,0.0,9.0,9.0,...,NaN,3.0,1.0,1.0,2.0,7.0,NaN,5.0,1.0,95.25
4,106304.0,2.000000,NaN,2.0,NaN,3.0,NaN,0.0,1.0,6.0,...,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.00


In [33]:
# Assuming `test_df_enriched` is your test dataset and `clf` is your trained LightGBM model

# Convert the 'ID' column from float to int
test_df_enriched['ID'] = test_df_enriched['ID'].astype(int)

# Separate out the 'ID' column and features from the 'test' dataset
test_features = test_df_enriched.drop('ID', axis=1)

# Impute the 'test' dataset using the fitted SimpleImputer
# Note: We use .transform() here, NOT .fit_transform()
test_imputed = imputer.transform(test_features)

# Predict the labels using the trained LightGBM model
predicted_labels = clf.predict(test_imputed)

# For binary classification, LightGBM might output probabilities. If so, convert these to 0 or 1
# Uncomment the following line if your model outputs probabilities
predicted_labels = np.round(predicted_labels).astype(int)

# Create a DataFrame with 'ID' and the predicted 'label'
results_df = pd.DataFrame({
    'ID': test_df_enriched['ID'],
    'LABEL': predicted_labels
})

# Optionally, convert the 'LABEL' column to the desired format, e.g., int
# results_df['LABEL'] = results_df['LABEL'].astype(int)

# Output the DataFrame with 'ID' and 'label'
# Save to a CSV file
results_df.to_csv('/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Submissions/my_submission_22_lightGBM_ADASYN_mdn_0.01LR.csv', index=False)

# Display the first few rows of the DataFrame
print(results_df.head())


       ID  LABEL
0   17547      0
1  140449      1
2  182658      1
3  149652      1
4  106304      1
